**환경**

In [1]:
import os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 안전: 토치 컴파일 끄기
os.environ["TORCH_COMPILE_DISABLE"] = "1"

MODEL_ID = "openai/gpt-oss-20b"

tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map=None,           # <- auto 금지
    low_cpu_mem_usage=False,   # <- 지연 로드 금지 (meta 방지)
)
model.to("cuda")               # <- 즉시 materialize
model.eval()

gen = pipeline("text-generation", model=model, tokenizer=tok)

# 짧게 구동 체크
out = gen([{"role": "user", "content": "안녕! 테스트야."}],
          max_new_tokens=40, do_sample=False)[0]["generated_text"]
print(out[-1]["content"])


c:\Users\Admin\anaconda3\envs\gptoss\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
You have loaded an FP4 model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model. To remove this warning, pass device_map = 'cuda'. 
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.73s/it]
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


analysisThe user says "안녕! 테스트야." which is Korean: "Hello! This is a test." They likely want a friendly response. The instruction: "You are ChatGPT


In [2]:
import json, re

SYSTEM = (
    "You are EmotionScore. Respond with JSON ONLY. "
    "Return exactly these four keys as floats in [0,1]: "
    "dopamine, serotonin, norepinephrine, melatonin. "
    "Do not write any other text, no explanation."
)

def emotion_vector(text: str):
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Text: {text}"},
        {"role": "assistant", "content": "{"}  # JSON 강제 시작
    ]
    out = gen(messages, max_new_tokens=60, do_sample=False,
              return_full_text=False)[0]["generated_text"]
    content = "{" + out

    m = re.search(r"\{.*\}", content, re.S)
    if not m:
        raise RuntimeError(f"JSON not found in: {content[:200]}")
    data = json.loads(m.group())

    def c(x): 
        try: return max(0.0, min(1.0, float(x)))
        except: return 0.0
    vec = [c(data.get(k)) for k in ("dopamine","serotonin","norepinephrine","melatonin")]
    return vec, data

# 테스트
demo = "우울하다"
vec, raw = emotion_vector(demo)
print("emotion vector [DA,5-HT,NE,MLT] =", vec)
print("raw =", raw)


emotion vector [DA,5-HT,NE,MLT] = [0.2, 0.3, 0.4, 0.6]
raw = {'dopamine': 0.2, 'serotonin': 0.3, 'norepinephrine': 0.4, 'melatonin': 0.6}


**감정 저장소**

In [3]:
from dataclasses import dataclass
from typing import Dict, Optional
import time

def clamp(x, lo=-1.0, hi=1.0):
    return max(lo, min(hi, x))

@dataclass
class EmotionState:
    DA: float = 0.0   # Dopamine
    HT: float = 0.0   # Serotonin
    NE: float = 0.0   # Noradrenaline
    ACh: float = 0.0  # Acetylcholine

    def clamp_(self):
        self.DA = clamp(self.DA); self.HT = clamp(self.HT)
        self.NE = clamp(self.NE); self.ACh = clamp(self.ACh)
        return self

from dataclasses import dataclass
from typing import Dict, Optional
import time

def clamp(x, lo=-1.0, hi=1.0):
    return max(lo, min(hi, x))

@dataclass
class EmotionState:
    DA: float = 0.0  # Dopamine
    HT: float = 0.0  # Serotonin
    NE: float = 0.0  # Noradrenaline
    ACh: float = 0.0  # Acetylcholine

    def clamp_(self):
        self.DA = clamp(self.DA); self.HT = clamp(self.HT)
        self.NE = clamp(self.NE); self.ACh = clamp(self.ACh)
        return self

class EmotionStore:
    def __init__(self, init: Optional[EmotionState] = None, decay_lambda: float = 0.9):
        self.raw = (init or EmotionState()).clamp_()
        self.decay_lambda = decay_lambda
        self.last_update_time = time.time()

    def update(self, delta: Dict[str, float]):
        """델타를 단순히 더해 저장"""
        self.raw.DA  = clamp(self.raw.DA  + delta.get("DA",  0.0))
        self.raw.HT  = clamp(self.raw.HT  + delta.get("HT",  0.0))
        self.raw.NE  = clamp(self.raw.NE  + delta.get("NE",  0.0))
        self.raw.ACh = clamp(self.raw.ACh + delta.get("ACh", 0.0))
        self.last_update_time = time.time()

    def get_state(self) -> EmotionState:
        """잔유 감쇠 계산해서 현재 상태 반환"""
        elapsed = max(1, int(time.time() - self.last_update_time))
        lam = self.decay_lambda ** elapsed
        return EmotionState(
            DA=clamp(self.raw.DA * lam),
            HT=clamp(self.raw.HT * lam),
            NE=clamp(self.raw.NE * lam),
            ACh=clamp(self.raw.ACh * lam),
        )
    
    # 👇 [수정/추가된 부분]
    def start_potential(self, K: float) -> float:
        """
        현재 감정 상태(기분)와 외부 자극(K)을 결합해 초기 전위를 계산합니다.
        - K: 현재 입력 텍스트의 감정 벡터 평균 (0~1)
        - mood: 현재 EmotionStore의 감쇠된 내부 상태
        """
        state = self.get_state()
        # 현재 기분을 단순 평균으로 계산 (-1~1 범위)
        mood = (state.DA + state.HT - state.NE + state.ACh) / 4.0

        # 초기 전위 = 70%의 외부 자극 + 30%의 현재 기분
        potential = (0.7 * K) + (0.3 * mood)
        
        # 최종값을 0과 1 사이로 클램핑하여 반환
        return max(0.0, min(1.0, potential))

**흥분성 뉴런**

In [4]:
def clamp(x, lo, hi): 
    return max(lo, min(hi, x))

class ExcitatoryNeuron:
    def __init__(self, weight=0.6, threshold=1.0, resting=0.0,
                 lr=0.05, w_bounds=(0.0, 2.0),
                 k_scale=1.2,         # ⬅ 입력 스케일 업
                 ht_coeff=0.2,        # ⬅ HT 임계 기여 약화 (기존 0.5)
                 ne_coeff=0.6):       # ⬅ NE 이득 강화 (기존 0.5)
        self.w = weight
        self.thr = threshold
        self.rest = resting
        self.vm = resting
        self.lr = lr
        self.w_lo, self.w_hi = w_bounds
        self.k_scale = k_scale
        self.ht_coeff = ht_coeff
        self.ne_coeff = ne_coeff

    def reset(self):
        self.vm = self.rest

    def forward_text(self, text: str, store=None):
        vec, _ = emotion_vector(text)          # [DA, 5HT, NE, MLT] in [0,1]
        DA, HT, NE, MLT = vec
        K = (DA + HT + NE + MLT) / 4.0
        x = (store.start_potential(K) if store is not None else K) * self.k_scale

        gain = 1.0 + self.ne_coeff * NE
        thr_eff = self.thr + self.ht_coeff * HT

        self.vm += self.w * x * gain

        fired = self.vm >= thr_eff
        y = 1.0 if fired else 0.0
        if fired:
            self.vm = self.rest

        self.w = clamp(self.w + self.lr * (DA - 0.5) * x, self.w_lo, self.w_hi)

        return y, {
            "w": round(self.w, 4),
            "vm": round(self.vm, 4),
            "fired": bool(fired),
            "gain": round(gain, 3),
            "thr_eff": round(thr_eff, 3)
        }


In [5]:
# test_excitatory.py
ex = ExcitatoryNeuron()

texts = [
    "나는 기쁘고 행복해",     # 긍정 문장
    "나는 우울하고 슬퍼",     # 부정 문장
    "나는 너가 불안하고 싫어" # 강한 부정 문장
]

print("=== ExcitatoryNeuron Test ===")
for t in texts:
    y, info = ex.forward_text(t)
    print(f"Text: {t}")
    print(f" Output y: {y}")
    for k, v in info.items():
        print(f"  {k}: {v}")
    print("-" * 30)


=== ExcitatoryNeuron Test ===
Text: 나는 기쁘고 행복해
 Output y: 0.0
  w: 0.6129
  vm: 0.5998
  fired: False
  gain: 1.36
  thr_eff: 1.16
------------------------------
Text: 나는 우울하고 슬퍼
 Output y: 0.0
  w: 0.6061
  vm: 0.9417
  fired: False
  gain: 1.24
  thr_eff: 1.06
------------------------------
Text: 나는 너가 불안하고 싫어
 Output y: 1.0
  w: 0.5985
  vm: 0.0
  fired: True
  gain: 1.48
  thr_eff: 1.06
------------------------------


**억제성 뉴런**

In [6]:
def clamp(x, lo, hi):
    return max(lo, min(hi, x))

class InhibitoryNeuron:
    """
    억제성 뉴런 (발화 튜닝 버전)
      - gain을 너무 깎지 않도록 하한↑
      - 세로토닌이 임계를 더 낮추도록 계수↑
      - 입력 스케일/초기 임계 약간 공격적으로
    """
    def __init__(self, weight=0.7, threshold=0.9, resting=0.0,
                 lr=0.04, w_bounds=(0.0, 2.0),
                 inhibit_strength=0.7,
                 k_scale=1.4,       # ↑ 입력 스케일
                 ht_coeff=0.35,     # ↑ 5-HT가 임계 낮추는 힘
                 ne_coeff=0.3,      # ↓ NE로 인한 억제 gain 감소 완화
                 gain_floor=0.6):   # ↑ gain 하한
        self.w = weight
        self.thr = threshold
        self.rest = resting
        self.vm = resting
        self.lr = lr
        self.w_lo, self.w_hi = w_bounds
        self.inhibit_strength = inhibit_strength
        self.k_scale = k_scale
        self.ht_coeff = ht_coeff
        self.ne_coeff = ne_coeff
        self.gain_floor = gain_floor

    def reset(self):
        self.vm = self.rest

    def _learn_w(self, x, DA, HT):
        d = 0.0
        d += (+0.90 if HT > 0.5 else -0.10) * self.lr * x
        d += (+0.70 if DA > 0.5 else -0.30) * self.lr * x
        self.w = clamp(self.w + d, self.w_lo, self.w_hi)

    def forward_text(self, text: str, store=None):
        vec, _ = emotion_vector(text)  # [DA, 5HT, NE, MLT]
        DA, HT, NE, MLT = vec
        K = (DA + HT + NE + MLT) / 4.0
        x = (store.start_potential(K) if store is not None else K) * self.k_scale

        gain = max(self.gain_floor, 1.0 - self.ne_coeff * NE)
        thr_eff = self.thr - self.ht_coeff * HT

        self.vm += self.w * x * gain

        fired = self.vm >= thr_eff
        y = -self.inhibit_strength if fired else 0.0
        if fired:
            self.vm = self.rest

        self._learn_w(x, DA, HT)

        return y, {
            "w": round(self.w, 4),
            "vm": round(self.vm, 4),
            "fired": bool(fired),
            "gain": round(gain, 3),
            "thr_eff": round(thr_eff, 3)
        }


In [7]:

inh = InhibitoryNeuron()

texts = [
    "나는 편안하고 차분해",     # 안정/평온
    "나는 짜증나고 화나",       # 분노/짜증
    "나는 무섭고 불안해",       # 불안/공포
    "나는 너가 불쾌하고 싫어"   # 강한 부정
]

print("=== InhibitoryNeuron Test ===")
for t in texts:
    y, info = inh.forward_text(t)   # store를 쓰지 않을 경우 None
    print(f"Text: {t}")
    print(f" Output y: {y}")        # 발화 시 음수(-inhibit_strength)
    for k, v in info.items():
        print(f"  {k}: {v}")        # w, vm, fired, gain, thr_eff
    print("-" * 30)


=== InhibitoryNeuron Test ===
Text: 나는 편안하고 차분해
 Output y: 0.0
  w: 0.7168
  vm: 0.4606
  fired: False
  gain: 0.94
  thr_eff: 0.62
------------------------------
Text: 나는 짜증나고 화나
 Output y: -0.7
  w: 0.7353
  vm: 0.0
  fired: True
  gain: 0.73
  thr_eff: 0.795
------------------------------
Text: 나는 무섭고 불안해
 Output y: 0.0
  w: 0.7258
  vm: 0.3325
  fired: False
  gain: 0.76
  thr_eff: 0.795
------------------------------
Text: 나는 너가 불쾌하고 싫어
 Output y: 0.0
  w: 0.7168
  vm: 0.6536
  fired: False
  gain: 0.79
  thr_eff: 0.795
------------------------------


**조절성 뉴런**

In [8]:
def clamp(x, lo, hi):
    return max(lo, min(hi, x))

class ModulatoryNeuron:
    """
    조절성 뉴런
      - emotion_vector(text) → [DA, 5HT, NE, MLT] ∈ [0,1]
      - 이 뉴런은 직접 발화 출력 대신, modulation dict 반환
        * dopamine   (DA): 보상/흥분 → 동기부여, 가중치 조절
        * serotonin  (HT): 안정/억제 → 발화 임계값 조절
        * norepineph (NE): 예민/각성 → 전체 gain 조절
        * acetylch.  (ACh): 집중/학습률 조절 (여기서는 MLT 대신 ACh로)
    """
    def __init__(self,
                 base_lr=0.05,
                 base_gain=1.0,
                 base_thr=1.0,
                 base_attn=1.0):
        self.base_lr = base_lr      # 기본 학습률
        self.base_gain = base_gain  # 기본 이득
        self.base_thr = base_thr    # 기본 임계
        self.base_attn = base_attn  # 기본 집중계수

    def forward_text(self, text: str):
        vec, _ = emotion_vector(text)   # [DA, 5HT, NE, MLT]
        DA, HT, NE, MLT = vec

        # 조절 파라미터 계산
        lr_mod   = self.base_lr   * (1.0 + 0.6 * DA)        # DA ↑ → 학습률 ↑
        thr_mod  = self.base_thr  * (1.0 + 0.5 * HT)        # 5-HT ↑ → 임계 ↑
        gain_mod = self.base_gain * (1.0 + 0.6 * NE)        # NE ↑ → 이득 ↑
        attn_mod = self.base_attn * (1.0 + 0.6 * MLT)       # MLT → 집중 ↑ (ACh 자리에 사용)

        mods = {
            "lr": round(lr_mod, 4),
            "thr": round(thr_mod, 4),
            "gain": round(gain_mod, 4),
            "attn": round(attn_mod, 4),
            "DA": round(DA,3), "HT": round(HT,3),
            "NE": round(NE,3), "MLT": round(MLT,3)
        }
        return mods


In [9]:
# test_modulatory.py
mod = ModulatoryNeuron()

texts = [
    "나는 기쁘고 즐겁다",   # 도파민↑
    "나는 안정되고 편안해", # 세로토닌↑
    "나는 불안하고 예민해", # 노르아드레날린↑
    "나는 집중해야 해"      # 아세틸콜린(집중)↑
]

print("=== ModulatoryNeuron Test ===")
for t in texts:
    mods = mod.forward_text(t)
    print(f"Text: {t}")
    for k, v in mods.items():
        print(f"  {k}: {v}")
    print("-"*30)


=== ModulatoryNeuron Test ===


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Text: 나는 기쁘고 즐겁다
  lr: 0.0755
  thr: 1.4
  gain: 1.42
  attn: 1.12
  DA: 0.85
  HT: 0.8
  NE: 0.7
  MLT: 0.2
------------------------------
Text: 나는 안정되고 편안해
  lr: 0.056
  thr: 1.4
  gain: 1.18
  attn: 1.42
  DA: 0.2
  HT: 0.8
  NE: 0.3
  MLT: 0.7
------------------------------
Text: 나는 불안하고 예민해
  lr: 0.059
  thr: 1.2
  gain: 1.42
  attn: 1.12
  DA: 0.3
  HT: 0.4
  NE: 0.7
  MLT: 0.2
------------------------------
Text: 나는 집중해야 해
  lr: 0.068
  thr: 1.2
  gain: 1.42
  attn: 1.12
  DA: 0.6
  HT: 0.4
  NE: 0.7
  MLT: 0.2
------------------------------


**신경망**

In [10]:
class OnePassNetwork:
    def __init__(self, store=None):
        self.store = store or EmotionStore()
        self.ex = ExcitatoryNeuron()
        self.inh = InhibitoryNeuron()
        self.mod = ModulatoryNeuron()

    def forward(self, text: str):
        vec, _ = emotion_vector(text)           # [DA, HT, NE, MLT]
        K = sum(vec) / len(vec)                 # 단순 평균
        x0 = self.store.start_potential(K)

        mods = self.mod.forward_text(text)      # lr, thr, gain, attn
        x = x0 * mods["attn"]

        yE, infoE = self.ex.forward_text(text, store=self.store)
        yI, infoI = self.inh.forward_text(text, store=self.store)

        # 델타 산출 (간단 규칙)
        delta = {
            "DA": +0.25*yE - 0.05*abs(yI),
            "HT": +0.20*abs(yI),
            "NE": +0.20*yE - 0.10*abs(yI),
            "ACh": +0.15*(mods["attn"]-1.0)
        }
        self.store.update(delta)

        return {
            "input": text,
            "vec": [round(v,3) for v in vec],
            "Vm0": round(x0,3),
            "mods": mods,
            "excitatory": infoE,
            "inhibitory": infoI,
            "delta": {k: round(v,3) for k,v in delta.items()},
            "state_after_update": self.store.raw.__dict__.copy()
        }


In [11]:
# response_control_neuro.py
# 전제: gen(...), emotion_vector(text) 존재
# 선택: OnePassNetwork가 있으면 vm0을 그쪽에서 받아 쓸 수 있음

def _clip01(x): 
    return max(0.0, min(1.0, float(x)))

def _affine(x, a=0.0, b=1.0):  # a + b*x
    return a + b*float(x)

def map_neuro_to_controls(vec, vm0: float = 0.0):
    """
    뉴로 파라미터(DA, 5HT, NE, MLT)와 막전위 vm0만으로 스타일 제어변수 산출.
    모든 변수는 0..1 범위로 클리핑. 감정 라벨은 일절 사용하지 않음.
    """
    DA, HT, NE, MLT = [_clip01(v) for v in vec]
    vm = max(0.0, float(vm0))

    # 순수 수학적 매핑 (설명어 없이)
    directness  = _clip01( 0.35 + 0.45*NE - 0.30*HT + 0.15*vm )
    brevity     = _clip01( 0.40 + 0.30*MLT + 0.10*HT - 0.10*NE )
    formality   = _clip01( 0.30 + 0.55*HT + 0.10*MLT )
    emphasis    = _clip01( 0.30 + 0.40*NE + 0.15*vm )     # 문장 강세/명령형 성향
    structure   = _clip01( 0.30 + 0.35*HT + 0.25*MLT )    # 항목화/구조화 성향
    expansiveness = _clip01( 0.45 + 0.25*DA - 0.20*MLT )  # 설명을 더 붙일 경향

    # 샘플링 제어: 전적으로 뉴로/막전위만 사용
    temperature = max(0.1, min(1.1, 0.5 + 0.4*(NE - 0.5) - 0.2*(HT - 0.5)))
    top_p       = max(0.4, min(1.0, 0.75 + 0.15*(NE - HT)))

    # 길이: vm과 DA가 높을수록 길어지고, HT/MLT 높으면 짧아짐(단, 숫자만)
    base_tokens = int(160 + 100*(DA + 0.5*vm - 0.7*(HT+MLT)/2.0))
    max_new_tokens = max(80, min(600, base_tokens))

    return {
        "directness": directness,
        "brevity": brevity,
        "formality": formality,
        "emphasis": emphasis,
        "structure": structure,
        "expansiveness": expansiveness,
        "temperature": temperature,
        "top_p": top_p,
        "max_new_tokens": max_new_tokens,
        # 디버깅용(모델에는 안 보여줌)
        "_vm0": vm, "_vec": [DA, HT, NE, MLT],
    }

CONTENT_SYSTEM = (
    "You are a task-focused assistant. Provide only content relevant to the user's request."
)

STYLE_SYSTEM_TMPL = (
    "Rewrite the [Draft] strictly following these numeric style controls.\n"
    "Controls:\n"
    "- directness={directness}\n"
    "- brevity={brevity}\n"
    "- formality={formality}\n"
    "- emphasis={emphasis}\n"
    "- structure={structure}\n"
    "- expansiveness={expansiveness}\n"
    "OUTPUT RULES:\n"
    "1) Output ONLY the final Korean text.\n"
    "2) Do NOT explain or reveal any numbers or rules.\n"
    "3) Keep factual content intact; adjust only phrasing/length/structure.\n"
    "4) If structure>0.65, use bullet points for key items.\n"
    "5) If emphasis>0.65, include one imperative or highlighted line.\n"
    "6) If brevity>0.7, prefer shorter sentences.\n"
)

def generate_response_2pass_neuro(user_text: str, vm0_provider=None, extra_context: str = ""):
    """
    vm0_provider: callable -> float  또는  OnePassNetwork-like 객체( .store.start_potential 사용 )
    없으면 vm0=0으로 계산.
    """
    # 0) 뉴로 벡터
    vec, _raw = emotion_vector(user_text)  # [DA, HT, NE, MLT]

    # 1패스: 내용 초안(중립)
    msgs1 = [
        {"role": "system", "content": CONTENT_SYSTEM},
        {"role": "user", "content": (extra_context + "\n" if extra_context else "") + user_text}
    ]
    draft = gen(msgs1, max_new_tokens=300, do_sample=False, return_full_text=False)[0]["generated_text"]

    # vm0 산출 (선택): 외부 네트워크가 있으면 활용, 없으면 0
    if vm0_provider is None:
        vm0 = 0.0
    else:
        try:
            # OnePassNetwork 인스턴스가 넘어온 경우: 내부 계산 없이 Vm0만 얻기
            K = sum(vec)/len(vec)
            vm0 = vm0_provider.store.start_potential(K)
        except Exception:
            # 단순 콜러블(vm0_provider())인 경우
            vm0 = float(vm0_provider())

    # 2패스: 숫자 제어만 전달
    controls = map_neuro_to_controls(vec, vm0)
    style_system = STYLE_SYSTEM_TMPL.format(**controls)
    msgs2 = [
        {"role": "system", "content": style_system},
        {"role": "user", "content": f"[Draft]\n{draft}"},
        {"role": "assistant", "content": ""}  # 최종문장만 강제
    ]
    final = gen(
        msgs2,
        max_new_tokens=controls["max_new_tokens"],
        do_sample=True,
        temperature=controls["temperature"],
        top_p=controls["top_p"],
        return_full_text=False
    )[0]["generated_text"]

    return {
        "draft": draft,
        "final": final,
        "controls": {k: v for k, v in controls.items() if not k.startswith("_")},
        "debug": {"vm0": controls["_vm0"], "vec": controls["_vec"]}
    }
